# DLITE Loss Function Modification
## Based on original loss function

In [26]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    # Define the Cross Entropy loss function here
    ce_loss_func = nn.CrossEntropyLoss()

    out = {}
    ce_out = {}
    model.eval()
    for split in ['train', 'val']:
        dlite_losses = torch.zeros(eval_iters)
        ce_losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, _ = model(X, Y)
            dlite_losses[k] = dlite_loss(logits, Y.view(-1)).item()
            ce_losses[k] = ce_loss_func(logits.view(-1, vocab_size), Y.view(-1)).item()
        out[split] = dlite_losses.mean()
        ce_out[split] = ce_losses.mean()
    model.train()
    return out, ce_out


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# Original version

class DLITELoss(nn.Module):
    def __init__(self):
        super(DLITELoss, self).__init__()

    def forward(self, logits, targets):
        # Convert logits to probabilities
        P = F.softmax(logits, dim=-1)
        
        # Convert targets to one-hot encoded format and ensure it's float type for operations
        Q = F.one_hot(targets, num_classes=vocab_size).float()

        # Masks for non-zero elements of P and Q
        mask_P = P > 0
        mask_Q = Q > 0

        # LIT(P, Q) part
        LIT_P = torch.zeros_like(P)
        LIT_Q = torch.zeros_like(Q)
        
        # Compute only for non-zero elements
        LIT_P[mask_P] = P[mask_P] * (1 - torch.log(P[mask_P]))
        LIT_Q[mask_Q] = Q[mask_Q] * (1 - torch.log(Q[mask_Q]))
        
        LIT_term = torch.sum(torch.abs(LIT_P - LIT_Q))

        # dH(P, Q) part
        dH_P = torch.zeros_like(P)
        dH_Q = torch.zeros_like(Q)
        
        # Compute only for non-zero elements
        dH_P[mask_P] = P[mask_P]**2 * (1 - 2*torch.log(P[mask_P]))
        dH_Q[mask_Q] = Q[mask_Q]**2 * (1 - 2*torch.log(Q[mask_Q]))
        
        dH_term = torch.sum(torch.abs(dH_P - dH_Q) / (2 * (P + Q)))

        # DLITE(P, Q)
        DLITE = LIT_term - dH_term
        
        return DLITE
'''
# Mask-operation improved DLITE loss
# Modified DLITE loss to accept logits and targets directly
class DLITELoss(nn.Module):
    def __init__(self):
        super(DLITELoss, self).__init__()

    def forward(self, logits, targets):
        # Convert logits to probabilities
        P = F.softmax(logits, dim=-1)
        
        # Convert targets to one-hot encoded format and ensure it's float type for operations
        Q = F.one_hot(targets, num_classes=vocab_size).float()

        # Masks for non-zero elements of P and Q
        mask_P = (P > 0).float()
        mask_Q = (Q > 0).float()

        # LIT(P, Q) part
        LIT_P = P * (1 - torch.log(P + 1e-9)) * mask_P
        LIT_Q = Q * (1 - torch.log(Q + 1e-9)) * mask_Q
        LIT_term = torch.sum(torch.abs(LIT_P - LIT_Q))

        # dH(P, Q) part
        dH_P = P**2 * (1 - 2 * torch.log(P + 1e-9)) * mask_P
        dH_Q = Q**2 * (1 - 2 * torch.log(Q + 1e-9)) * mask_Q
        dH_term = torch.sum(torch.abs(dH_P - dH_Q) / (2 * (P + Q + 1e-9)))

        # DLITE(P, Q)
        DLITE = LIT_term - dH_term
        
        return DLITE
'''
    
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.loss_func = DLITELoss()  # Initialize the DLITE loss function

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = self.loss_func(logits, targets)  # Use DLITE loss here

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx



The baseline result shows a very large loss.

In [27]:
# hyperparameters
batch_size = 16
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

torch.manual_seed(1337)


model = BigramLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# Create an instance of the custom DLITE loss class
dlite_loss = DLITELoss()

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        dlite_losses, ce_losses = estimate_loss()
        print(f"step {iter}: train loss {dlite_losses['train']:.4f}, val loss {dlite_losses['val']:.4f}, CE train loss: {ce_losses['train']:.4f}, CE val loss {ce_losses['val']:.4f}\n")

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the logits
    logits, _ = model(xb, yb)

    # Flatten the logits and target tensor
    logits_flat = logits.view(-1, logits.shape[-1])
    targets_flat = yb.view(-1)

    # Compute the DLITE loss using the flattened logits and targets
    loss = dlite_loss(logits_flat, targets_flat)

    # Backpropagation and optimization step
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 475.1874, val loss 474.8864, CE train loss: 4.4116, CE val loss 4.4022

step 100: train loss 344.2476, val loss 344.8734, CE train loss: 3.3289, CE val loss 3.4023

step 200: train loss 331.4117, val loss 332.6119, CE train loss: 3.3358, CE val loss 3.4170

step 300: train loss 323.1970, val loss 326.0095, CE train loss: 3.3310, CE val loss 3.4452

step 400: train loss 316.9828, val loss 319.1192, CE train loss: 3.2579, CE val loss 3.3511

step 500: train loss 310.5850, val loss 313.5168, CE train loss: 3.2304, CE val loss 3.3182

step 600: train loss 302.1467, val loss 302.7675, CE train loss: 3.0796, CE val loss 3.1085

step 700: train loss 297.8466, val loss 300.0448, CE train loss: 3.0905, CE val loss 3.1318

step 800: train loss 290.9451, val loss 294.6096, CE train loss: 3.0216, CE val loss 3.0721

step 900: train loss 287.2826, val loss 290.7303, CE train loss: 2.9895, CE val loss 3.0326

step 1000: train loss 284.9845, val loss 288.5309,

## Based on Lixiao's previous version
The following code is modified based on Prof. Ke's version of DLITE loss function, I used multification instead of indexing for the mask to improve the calculation speed.

In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

ce_loss_func = nn.CrossEntropyLoss()
dlite_loss_func = DLITELoss()

@torch.no_grad()
def estimate_loss():
    dlite_out = {}
    ce_out = {}
    model.eval()
    for split in ['train', 'val']:
        dlite_losses = torch.zeros(eval_iters)
        ce_losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, _ = model(X, Y)
            
            # Compute DLITE loss
            dlite_losses[k] = dlite_loss_func(logits.view(-1, vocab_size), Y.view(-1)).item()
            
            # Compute cross-entropy loss
            ce_losses[k] = ce_loss_func(logits.view(-1, vocab_size), Y.view(-1)).item()

        dlite_out[split] = dlite_losses.mean()
        ce_out[split] = ce_losses.mean()
    model.train()
    return dlite_out, ce_out


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x
'''
def one_hot_encode(indices, num_classes):
    """Converts indices (LongTensor) into a one-hot encoded tensor."""
    one_hot = torch.zeros(indices.shape[0], num_classes, device=indices.device)
    one_hot.scatter_(1, indices.unsqueeze(-1), 1)
    return one_hot
'''    
class DLITELoss(nn.Module):
    def __init__(self):
        super(DLITELoss, self).__init__()

    def forward(self, logits, targets, epsilon=1e-10):
        # Convert logits to probabilities using softmax
        probs = F.softmax(logits, dim=-1)
        
        # Convert targets to one-hot encoded format and ensure it's float type for operations
        Q = F.one_hot(targets, num_classes=vocab_size).float()
        P = probs
        
        # Masks for non-zero elements of P and Q
        mask_P = (P > 0).float()
        mask_Q = (Q > 0).float()

        # LIT(P, Q) part
        LIT_P = P * (1 - torch.log(P + epsilon)) * mask_P
        LIT_Q = Q * (1 - torch.log(Q + epsilon)) * mask_Q
        LIT_term = torch.sum(torch.abs(LIT_P - LIT_Q))

        # dH(P, Q) part
        dH_P = P**2 * (1 - 2 * torch.log(P + epsilon)) * mask_P
        dH_Q = Q**2 * (1 - 2 * torch.log(Q + epsilon)) * mask_Q
        dH_term = torch.sum(torch.abs(dH_P - dH_Q) / (2 * (P + Q + epsilon)))

        # DLITE(P, Q)
        DLITE = LIT_term - dH_term
        
        return DLITE

    
# super simple bigram model
class DLITEBigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.loss_func = DLITELoss()  # Initialize the DLITE loss function

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = self.loss_func(logits, targets)  # Use DLITE loss here

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx



In [11]:
# hyperparameters
batch_size = 16
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

torch.manual_seed(1337)

model = DLITEBigramLanguageModel().to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        dlite_losses, ce_losses = estimate_loss()
        print(f"step {iter}: train loss {dlite_losses['train']:.4f}, val loss {dlite_losses['val']:.4f}, CE train loss: {ce_losses['train']:.4f}, CE val loss {ce_losses['val']:.4f}\n")

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the logits
    logits, _ = model(xb, yb)

    # Flatten the logits and target tensor
    logits_flat = logits.view(-1, logits.shape[-1])
    targets_flat = yb.view(-1)

    # Compute the DLITE loss using the flattened logits and targets
    loss = dlite_loss_func(logits_flat, targets_flat)

    # Backpropagation and optimization step
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Generate some sample text
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 475.1874, val loss 474.8864, CE train loss: 4.4116, CE val loss 4.4022

step 100: train loss 344.2480, val loss 344.8739, CE train loss: 3.3289, CE val loss 3.4023

step 200: train loss 331.4091, val loss 332.6132, CE train loss: 3.3353, CE val loss 3.4166

step 300: train loss 323.1790, val loss 326.0051, CE train loss: 3.3310, CE val loss 3.4455

step 400: train loss 316.9781, val loss 319.1335, CE train loss: 3.2563, CE val loss 3.3502

step 500: train loss 310.6371, val loss 313.5950, CE train loss: 3.2320, CE val loss 3.3204

step 600: train loss 302.1804, val loss 302.8387, CE train loss: 3.0792, CE val loss 3.1113

step 700: train loss 297.7800, val loss 299.8920, CE train loss: 3.0864, CE val loss 3.1272

step 800: train loss 290.9508, val loss 294.5722, CE train loss: 3.0211, CE val loss 3.0722

step 900: train loss 287.5330, val loss 290.9762, CE train loss: 2.9829, CE val loss 3.0255

step 1000: train loss 284.7902, val loss 288.3075,

## Conclusion

Both versions shows large loss using DLITE, the following part needs to be examined to address potential issue:
- One-hot transformation causing dimentionality increase (line 150)